In [11]:
import yaml
import json
from openai import OpenAI
import sqlite3
import pandas as pd

In [14]:
with open('api_keys.yaml', 'r') as f:
    api_keys = yaml.safe_load(f)

OPENAI_API_KEY = api_keys['openai-key']
client = OpenAI(api_key=OPENAI_API_KEY)

## Define tools and functions
Need functions to determine what tables to call. Predefined table structure

In [6]:
db_name = "lldm.db"
try: 
    db = sqlite3.connect(db_name) 
    print('Connected to DB')
except: 
    print("DB connection failed")

Connected to DB


In [7]:
# run query and return pandas dataframe
def run_query(db, query):
    try:
        df = pd.read_sql_query(query, db)
        return df
    # not a select statement
    except TypeError:
        cursor = db.cursor()
        cursor.execute(query)
        return    

In [8]:
sql_query = """
SELECT name FROM sqlite_master  
WHERE type='table';
"""
cursor = db.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('CHARACTER_SHEET',), ('INVENTORY',), ('SETTINGS',), ('NPCS',), ('TREASURES',), ('MONSTERS',), ('PLOT',), ('LOGS',), ('CAMPAIGN',), ('CHARACTER_INVENTORY',)]


In [12]:
query = '''
SELECT * FROM INVENTORY;
'''
run_query(db,query)

,Item_ID,Category,Item,Description
0,0,Weapon,Longsword,Elara's main weapon for combat.
1,1,Weapon,Dagger,Versatile tool for close combat and utility pu...
2,2,Adventuring Gear,Backpack,To carry essential items.
3,3,Adventuring Gear,Rope (50 feet),Useful for climbing and securing objects.
4,4,Adventuring Gear,Rations (5 days),Enough food for the journey.
5,5,Adventuring Gear,Water skin,To carry water.
6,6,Adventuring Gear,Flint and Steel,For starting fires.
7,7,Adventuring Gear,Healing Potions (2),For quick recovery in emergencies.
8,8,Armor,Chain Mail,Provides solid protection while allowing mobil...
9,9,Armor,Shield,Additional defense against attacks.


In [13]:
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity


### Picking up item
For now, this is the pipeline:
- Item must come from INVENTORY table in db
- Assume that the item will always be written exactly as it appears in the table
- 

In [56]:
# check if item queried is in INVENTORY table
def validate_item(item_name):
    query = f'''
    SELECT * FROM INVENTORY WHERE UPPER(Item) LIKE "{item_name.upper()}%" 
    '''
    tmp = run_query(db, query)
    if not tmp.empty:
        return tmp['Item_ID'].iloc[0]
    return None

# update table if item validated, otherwise error message
# for now, use temporary campaign and character id
def get_obtained_item(item_name, quantity, campaign_id=0, character_id=0):
    item_id = validate_item(item_name)
    print(item_id, quantity)
    if item_id is not None:
        query = f'''
        IF EXISTS (SELECT * FROM CHARACTER_INVENTORY WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id})
            UPDATE CHARACTER_INVENTORY SET (Quantity = Quantity + 1) 
            WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id}
        ELSE
            INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity)
            VALUES ({campaign_id}, {character_id}, {item_id}, {quantity})
        '''
        run_query(db, query) # TODO: ERROR HANDLING
        return json.dumps({'message':'The item(s) were successfully obtained. Please continue the story.'})
    else:
        return json.dumps({'message':'Item does not exist. Please prompt user to specify further or provide another action.'})


def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "I pick up a longsword from the bodies of the dead goblins."}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_obtained_item",
                "description": "Extract the item that the user has obtained in some manner (such as picked up, purchased, etc.)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "item_name": {
                            "type": "string",
                            "description": "The name of the obtained item.",
                        },
                        "quantity": {
                            "type": "integer",
                            "description": "The number of said items obtained. If a number is not specified, try and infer based on the surrounding context."
                        }
                    },
                    "required": ["item_name", "quantity"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        #tool_choice="auto",  # auto is default, but we'll be explicit
        tool_choice = {"type":'function','function':{'name':'get_obtained_item'}}
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_obtained_item": get_obtained_item,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                item_name=function_args.get("item_name"),
                quantity=function_args.get("quantity"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


In [57]:
print(run_conversation())

0 1


DatabaseError: Execution failed on sql '
        IF EXISTS (SELECT * FROM CHARACTER_INVENTORY WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0)
            UPDATE CHARACTER_INVENTORY SET (Quantity = Quantity + 1) 
            WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0
        ELSE
            INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity)
            VALUES (0, 0, 0, 1)
        ': near "IF": syntax error

In [54]:
# Check to see if tables were updated
# TODO: WHY IS THIS UPDATED TWICE??
query = '''
SELECT * FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

,Campaign_ID,Character_ID,Item_ID,Quantity


In [53]:
query = '''
DELETE FROM CHARACTER_INVENTORY;
'''
run_query(db,query)

In [61]:
query = '''
SELECT
CASE
    WHEN EXISTS(SELECT 1 FROM CHARACTER_INVENTORY WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0)
    THEN 
        UPDATE CHARACTER_INVENTORY SET (Quantity = Quantity + 1) 
        WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0
    ELSE 
        INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity)
        VALUES (0, 0, 0, 1)
END;
'''
run_query(db, query)

DatabaseError: Execution failed on sql '
SELECT
CASE
    WHEN EXISTS(SELECT 1 FROM CHARACTER_INVENTORY WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0)
    THEN 
        UPDATE CHARACTER_INVENTORY SET (Quantity = Quantity + 1) 
        WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0
    ELSE 
        INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity)
        VALUES (0, 0, 0, 1)
END;
': near "UPDATE": syntax error

In [60]:
query = '''
IF EXISTS (SELECT * FROM CHARACTER_INVENTORY WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0)
    UPDATE CHARACTER_INVENTORY SET (Quantity = Quantity + 1) 
    WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0
ELSE
    INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity)
    VALUES (0, 0, 0, 1)
'''
run_query(db, query)

DatabaseError: Execution failed on sql '
IF EXISTS (SELECT * FROM CHARACTER_INVENTORY WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0)
    UPDATE CHARACTER_INVENTORY SET (Quantity = Quantity + 1) 
    WHERE Item_ID = 0 AND Campaign_ID = 0 AND Character_ID = 0
ELSE
    INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Quantity)
    VALUES (0, 0, 0, 1)
': near "IF": syntax error